In [ ]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata

## Laden der Daten aus Datei: 

In [ ]:
# open from file: 
with open("pandemie_titeldaten_oa.xml", "r", encoding="utf-8") as f2:
    result = f2.readlines()

In [ ]:
#print(result[0:500])

Parsen des XML aus Datei:

In [ ]:
content = "".join(result)
#print(content[0:500])

In [ ]:
xml = soup(content, "lxml")

In [ ]:
#print(xml.prettify()[:500])

Heraussuchen aller Datensätze des Typs "Bibliographic" aus dem XML:

In [ ]:
myquery = xml.find_all('record', {'type':'Bibliographic'})
print(len(myquery))

In [ ]:
#print(myquery[1].prettify())


## Suche nach benötigten Informationen in den einzelnen Datensätzen: 

In [ ]:
#Funktion für Titeldaten in MARC21

def parse_record_marc(item):

    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(item)))
    
    #idn
    idn = xml.findall("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'N/A' 
        
    
    #Titel 
    title = xml.findall("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    title2 = xml.findall("marc:datafield[@tag = '245']/marc:subfield[@code = 'b']", namespaces=ns)
    
    if title and not title2:
        titletext = title[0].text
    elif title and title2:     
        titletext = title[0].text + " " + title2[0].text   #Hier wird extra nur mit einem Leerzeichen getrennt statt eines Separators
    else:
        titletext = "N/A"
    
    
    #date
    date = xml.findall("marc:datafield[@tag = '264']/marc:subfield[@code = 'c']", namespaces=ns)
    date2 = xml.findall("marc:controlfield[@tag = '008']", namespaces=ns)
    if date:
        date = date[0].text
    elif date2:
        date = date2[0].text
        date = date[7:11]
    else:    
        date = 'N/A'
          
    #lang:
    lang = xml.findall("marc:datafield[@tag = '041']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        lang = lang[0].text
    except:
        lang = 'N/A'
    
    
    #URN: 
    urn = "N/A"
    for urn in xml.findall("marc:datafield[@tag = '024']/marc:subfield[@code = 'a']", namespaces=ns):
        if urn.text.startswith("urn:nbn"):
            urn = urn.text
        else: 
            urn = "N/A"
    
    
    meta_dict = {"idn":idn, "title": titletext, "date":date, "lang":lang, "urn":urn} 
               
    return meta_dict
    

In [ ]:
output = [parse_record_marc(record) for record in myquery]

In [ ]:
df = pd.DataFrame(output)
df

In [ ]:
df.to_hdf("pandemie_titeldaten_oa.h5", key="df")